In [3]:
import pandas as pd
trainDf = pd.read_csv('./train_data', encoding='gb2312')

In [4]:
trainDf.head()

,id,account,group,IP,url,port,vlan,switchIP,time,ret
0,1,xiaojiawei@qq.com,人事行政中心,192.168.1.50,http://123.6.4.41,15788,700,129.30.06.37,2021/6/16 7:56,0.1149
1,2,xiaojiawei@qq.com,人事行政中心,192.168.31.46,http://104.192.108.154,12665,700,152.91.89.45,2021/6/28 7:58,0.1801
2,3,xiaojiawei@qq.com,人事行政中心,192.168.2.3,http://42.236.37.80,25551,700,129.30.06.37,2021/6/1 6:37,0.3690
3,4,xiaojiawei@qq.com,人事行政中心,192.168.145.30,http://uf-api6-client.ksord.com,18274,700,162.6.8.29,2021/5/5 8:18,0.1532
4,5,xiaojiawei@qq.com,人事行政中心,192.168.178.92,http://101.199.128.170,34572,700,162.0.1.84,2021/6/20 6:52,0.1449


In [84]:
trainDf.shape

(528690, 10)

In [11]:
trainDf['account'].unique()

array(['xiaojiawei@qq.com', 'wangyang@qq.com', 'liyuan@qq.com',
       'liulin@qq.com', 'xuemiaomiao@qq.com', 'wangliujie@qq.com',
       'lipanpan@qq.com', 'liyang@qq.com', 'anpeng@qq.com',
       'yangjing@qq.com', 'wangshibiao@qq.com', 'liuyanlong@qq.com',
       'xuyao@qq.com', 'wangyang1@qq.com', 'yanghao@qq.com',
       'xudongmei@qq.com', 'dairan@qq.com', 'xiexiaojing@qq.com',
       'libin@qq.com', 'lixiaojing@qq.com', 'jingai@qq.com',
       'wangqianying@qq.com', 'lianghongyu@qq.com', 'chenfu@qq.com',
       'sunwei@qq.com', 'chenge@qq.com', 'qinlilil@qq.com',
       'wangwei@qq.com', 'machengcheng@qq.com', 'zhangxingqi@qq.com',
       'lisong@qq.com', 'liuyiyao@qq.com', 'haoxiaofang@qq.com',
       'jibin@qq.com', 'wanglicai@qq.com', 'jiangwei@qq.com',
       'chenjie@qq.com', 'guowenjian@qq.com', 'quanxiaoru@qq.com',
       'wangjiani@qq.com', 'wangyu@qq.com', 'xuxinyu@qq.com',
       'kangjunwei@qq.com', 'libingkun@qq.com', 'taolan@qq.com',
       'zhanghong@qq.com', 'qiul

In [14]:
group = trainDf['group'].unique()
print(group)

['人事行政中心' '业务创新中心' '市场战略发展中心' '政企事业部' '研发中心' '渠道生态合作事业部' '通用市场部']


In [17]:
groupOneHot = pd.get_dummies(trainDf['group'])

In [62]:
import requests
import time
def geturlInfo(url):
    server='http://ip-api.com/json/'
    time.sleep(0.0001)
    response = requests.get(server+url,headers={'User-agent': 'Firefox'})
    sleeptime = 10
    while response.status_code==429:
        if sleeptime>80:
            print('warning')
        time.sleep(sleeptime)
        sleeptime *= 2
        response = requests.get(server+url,headers={'User-agent': 'Firefox'})
    body = response.json()
    return pd.Series([body.get('countryCode'),body.get('lat'),body.get('lon')],index=['country','lat','lon'])

In [30]:
import urllib
geturlInfo(re.match('.*//()','http://uf-api6-client.ksord.com'))

(None, None, None)

In [37]:
urlSelect = trainDf['url'].apply(lambda x: re.match('^.*//(.*)$',x).group(1))

In [43]:
import numpy as np
np.where(urlSelect.isnull())

(array([], dtype=int64),)

In [87]:
import tqdm
tqdm.__version__

'4.54.1'

In [88]:
import geoip2.database
reader = geoip2.database.Reader('./GeoLite2-City_20210921/GeoLite2-City.mmdb')

def geturlInfo(url):
    if re.match('^((2(5[0-5]|[0-4]\d))|[0-1]?\d{1,2})(\.((2(5[0-5]|[0-4]\d))|[0-1]?\d{1,2})){3}$', url):
        if privateIp(url)!='D':
            return 1
        return int(reader.city(url).country.iso_code=='CN')
    else:
        try:
            ip = socket.gethostbyname(url)
            return int(reader.city(ip).country.iso_code=='CN')
        except:
            return 1
urlCountry = urlSelect.apply(lambda x: geturlInfo(x))

In [90]:
urlCountry.unique()

array([1, 0])

In [83]:
urlinfo = np.empty((urlSelect.shape[0],),dtype=[('country',np.bytes_),('lat',float),('lon',float)])
for i in tqdm(range(urlSelect.shape[0])):
    temp = geturlInfo(urlSelect[i]).to_numpy()
    urlinfo[i]['country'] = temp[0]
    urlinfo[i]['lat'] = temp[1]
    urlinfo[i]['lon'] = temp[2]

KeyboardInterrupt: 

In [82]:
geturlInfo(urlSelect[i]).to_numpy()

array(['CN', 33.4408, 113.608], dtype=object)

In [73]:
#https://stackoverflow.com/questions/18603270/progress-indicator-during-pandas-operations
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
urlinfo = pd.DataFrame({'url': urlSelect}).progress_apply(lambda x: geturlInfo(x))

AttributeError: 'DataFrame' object has no attribute '_is_builtin_func'

In [61]:
%debug

> /opt/gentoo/usr/lib/python3.8/site-packages/simplejson/decoder.py(400)raw_decode()
    396             if ord0 == 0xfeff:
    397                 idx += 1
    398             elif ord0 == 0xef and s[idx:idx + 3] == '\xef\xbb\xbf':
    399                 idx += 3
--> 400         return self.scan_once(s, idx=_w(s, idx).end())



ipdb>  u


> /opt/gentoo/usr/lib/python3.8/site-packages/simplejson/decoder.py(370)decode()
    368         if _PY3 and isinstance(s, bytes):
    369             s = str(s, self.encoding)
--> 370         obj, end = self.raw_decode(s)
    371         end = _w(s, end).end()
    372         if end != len(s):



ipdb>  u


> /opt/gentoo/usr/lib/python3.8/site-packages/simplejson/__init__.py(525)loads()
    523             parse_constant is None and object_pairs_hook is None
    524             and not use_decimal and not kw):
--> 525         return _default_decoder.decode(s)
    526     if cls is None:
    527         cls = JSONDecoder



...

ipdb>  d


> <ipython-input-59-9cef9ad5acab>(10)geturlInfo()
      7     if response.status_code==429:
      8         time.sleep(10)
      9         response = requests.get(server+url,headers={'User-agent': 'Firefox'})
---> 10     body = response.json()
     11     return pd.Series([body.get('countryCode'),body.get('lat'),body.get('lon')],index=['country','lat','lon'])



ipdb>  response


<Response [429]>


ipdb>  q


In [ ]:
urlinfo.head()

In [19]:
# https://serverfault.com/questions/52631/is-it-better-to-use-the-192-168-x-x-or-10-x-x-x-address-range-for-a-small-busine
import re
def privateIp(ip):
    if re.match('^192.168..*',ip):
        return 'C'
    elif re.match('^10..*',ip):
        return 'A'
    elif re.match('^172..*', ip):
        return 'B'
    else:
        return 'D'

In [21]:
trainDf['IP'].apply(lambda x: privateIp(x)).unique()

array(['C', 'A'], dtype=object)

In [22]:
ipOneHot = pd.get_dummies(trainDf['IP'].apply(lambda x: privateIp(x)))

In [91]:
trainDf['time'].dtype

dtype('O')

In [98]:
import datetime
times = datetime.datetime.strptime('2021/6/16 7:56','%Y/%m/%d %H:%M')
print(times.hour,times.minute,times.weekday())

7 56 2


In [105]:
type(trainDf['time'])

pandas.core.series.Series

In [106]:
timeInfo = trainDf['time'].apply(lambda x: pd.Series([datetime.datetime.strptime(x,'%Y/%m/%d %H:%M').hour+datetime.datetime.strptime(x,'%Y/%m/%d %H:%M').minute/60-12,int(times.weekday()<=6)], index=['hours','weekday']))

In [110]:
trainX = timeInfo.join(urlCountry)

In [115]:
trainY = trainDf['ret']

In [111]:
trainX

,hours,weekday,url
0,-4.066667,1.0,1
1,-4.033333,1.0,1
2,-5.383333,1.0,1
3,-3.700000,1.0,1
4,-5.133333,1.0,1
...,...,...,...
528685,0.383333,1.0,1
528686,-0.450000,1.0,1
528687,-1.716667,1.0,1
528688,-0.916667,1.0,1


In [116]:
import h5py
with h5py.File('train.h5','w') as opt:
    opt.create_dataset('x', data=trainX.to_numpy(), compression='gzip')
    opt.create_dataset('y', data=trainY.to_numpy(), compression='gzip')    

In [119]:
def transfer(ipt='./A_test_data.csv', outfile='test.h5',y=False):
    testDf = pd.read_csv(ipt, encoding='gb2312')
    urlSelect = testDf['url'].apply(lambda x: re.match('^.*//(.*)$',x).group(1))
    urlCountry = urlSelect.apply(lambda x: geturlInfo(x))
    timeInfo = testDf['time'].apply(lambda x: pd.Series([datetime.datetime.strptime(x,'%Y/%m/%d %H:%M').hour+datetime.datetime.strptime(x,'%Y/%m/%d %H:%M').minute/60-12,int(times.weekday()<=6)], index=['hours','weekday']))    
    testX = timeInfo.join(urlCountry)
    if y:
        testY = testDf['ret']
    with h5py.File(outfile, 'w') as opt:
        opt.create_dataset('x', data=testX.to_numpy(), compression='gzip')
        if y:
            opt.create_dataset('y', data=testY.to_numpy(), compression='gzip')

In [120]:
transfer()